In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [6]:
# Load cleaned data
df = pd.read_csv("diabetes_cleaned.csv")

In [8]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

In [10]:
# Handle imbalanced data
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [12]:
# Feature Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [16]:
# Calculate scale_pos_weight to handle imbalance
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)

In [18]:
# Define XGBoost model
xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    scale_pos_weight=scale_pos_weight
)

In [32]:
# Define hyperparameter grid
param_grid = {
    "n_estimators": [200, 500],
    "learning_rate": [0.01, 0.02, 0.05],
    "max_depth": [4, 5, 6],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9],
    "reg_alpha": [0, 0.01, 0.1],
    "reg_lambda": [0.1, 1, 10]
}

In [31]:
# Perform Grid Search
grid_search = GridSearchCV(xgb, param_grid, cv=3, scoring="accuracy", verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 1458 candidates, totalling 4374 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'learning_rate': [0.01, 0.02, 0.05],
                         'max_depth': [4, 5, 6], 'n_estimators': [200, 500],
                         'reg_alpha': [0, 0.01, 0.1],
                         'reg_lambda': [0.1, 1, 10],
                         'subsample': [0.7, 0.8, 0.9]},
             scoring='accuracy', verbose=1)

In [24]:
# Get best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 500, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.9}


In [25]:
# Predict and evaluate
y_pred = best_model.predict(X_test)

In [26]:
# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [48]:
print(f"XGBoost Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

XGBoost Accuracy: 0.8150
Precision: 0.7944, Recall: 0.8500, F1-score: 0.8213
Confusion Matrix:
[[78 22]
 [15 85]]
